# Read Bonn Data


In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from heapq import heappush, heappop, heapify
from collections import defaultdict, Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/My Drive/0-Project')

In [4]:
folders = ['F', 'O', 'S']
dataframes = {}

for folder_path in folders:
  file_list = os.listdir(folder_path)
  data_dict = {}

  for file_name in file_list:
    column_name = file_name[1:4]
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
      file_data = [line.strip() for line in file.readlines()]
      data_dict[column_name] = file_data

  df = pd.DataFrame(data_dict)
  df = df[sorted(df.columns)]

  # Store the DataFrame in the dictionary with the folder name as the key
  dataframes[folder_path] = df

# Accessing the DataFrames for F, O, and S
df_F = dataframes['F']
df_O = dataframes['O']
df_S = dataframes['S']

# Print head and shape of each DataFrame
print("DataFrame for F:")
print(df_F.head())
print(df_F.shape)  # Should be (4097, 100)

print("\nDataFrame for O:")
print(df_O.head())
print(df_O.shape)  # Should be (4097, 100)

print("\nDataFrame for S:")
print(df_S.head())
print(df_S.shape)  # Should be (4097, 100)


DataFrame for F:
  001 002 003  004  005  006  007 008   009 010  ...  091   092  093  094  \
0  34  60  26  -41   13  -15  -24  23  -263  59  ...   99  -131    1  -41   
1  33  47  16  -42    6   -2  -27  17  -263  52  ...  114  -153   -4  -41   
2  28  38  13  -48   -1    0  -23  10  -261  51  ...  122  -177   -6  -48   
3  22  29  12  -48  -13    2  -28  10  -258  46  ...  132  -194  -15  -48   
4  21  28  17  -48  -29   -2  -34   7  -258  43  ...  151  -204   -8  -44   

   095  096 097  098 099  100  
0  -39   45  75   67   5  -45  
1  -27   52  72   86   2  -53  
2  -16   79  72   99  -6  -51  
3   -3  117  80  109  -4  -52  
4   17  146  81  115  -9  -54  

[5 rows x 100 columns]
(4097, 100)

DataFrame for O:
   001  002  003  004  005  006  007  008  009  010  ...  091   092   093  \
0  -24  -55  -36  -14  -58   87  -52    2    8  -53  ...   62  -128   -83   
1  -22  -48  -40   -5  -78   98  -56   -6    0  -15  ...   49  -158  -120   
2  -17  -48  -36    0  -83  103  -49  -22  

In [5]:
df_F = df_F.apply(pd.to_numeric)
df_O = df_O.apply(pd.to_numeric)
df_S = df_S.apply(pd.to_numeric)

# Implement DPCM and Huffman compression algorithm

Example:

We have a series of data:

100 70 120 40 50 80

We calculate the difference between them (DPCM):

100 -30 50 -80 10 30

Then for all these new data, we use Huffman encoding to compress them


Since we are using the difference, we don't have to add offset to original data.

When we are dealing with a dataframe, should we flatten the dataframe into a list or we preserve the value of first row for each column?

Each column represents a sample of data series. If we flatten the data into a list, we assume that all data form a series. But that's not the case. So we preserve the value of first row.

In [7]:
def DPCM(data):
  '''
  Params:
    data : input data frame
  For each column, calculate the difference between i-th row and i+1-th row,
  use this value to represent the data in i+1-th row.
  This function preserves the value in first row of each column.
  Returns a new dataframe.
  '''
  temp = data.copy()
  for col in temp.columns:
    diff_res = temp[col].diff()
    diff_res.iloc[0] = temp[col].iloc[0]
    temp[col] = diff_res
  return temp

When doing Huffman coding, we can flatten the dataframe into a list. In reality we might need to compress and send many data samples at a time, these samples are not from one series. If we calculate one Huffman tree for each column, then we need many frequency lists, which is not efficient.

In [14]:
class HuffmanNode:
  '''
  A Huffman tree node.
  '''
  def __init__(self, value=None, frequency=None, left=None, right=None):
    self.value = value
    self.frequency = frequency
    self.left = left
    self.right = right

  # Comparison methods for the priority queue
  def __lt__(self, other):
    return self.frequency < other.frequency

def build_huffman_tree(frequency):
  '''
  Params:
    frequency : dictionary of frequencies
  Returns the root of a Huffman tree.
  '''
  heap = [HuffmanNode(value, freq) for value, freq in frequency.items()]
  heapify(heap)

  while len(heap) > 1:
    left = heappop(heap)
    right = heappop(heap)
    merged = HuffmanNode(None, left.frequency + right.frequency, left, right)
    heappush(heap, merged)

  return heap[0]

def generate_codes(node, prefix="", codebook={}):
  '''
  Params:
    node : Huffman tree node
    prefix : prefix code
    codebook : dictionary of codes
  Returns a dictionary of codes.
  '''
  if node is not None:
    if node.value is not None:
      codebook[node.value] = prefix
    generate_codes(node.left, prefix + "0", codebook)
    generate_codes(node.right, prefix + "1", codebook)
  return codebook

def huffman_encoding(data):
  '''
  Params:
    data : input data frame
  Returns a tuple of encoded data and Huffman codes.
  '''
  # flatten the DataFrame and calculate frequencies
  values = data.values.flatten()
  frequency = Counter(values)

  # build Huffman Tree
  huffman_tree = build_huffman_tree(frequency)

  # generate Huffman Codes
  huffman_codes = generate_codes(huffman_tree)

  # encode the dataFrame
  encoded_data = data.applymap(lambda x: huffman_codes[x])

  return encoded_data, huffman_codes

#Test case

In [15]:
# Test huffman tree
data = pd.DataFrame({
    'A': [1, 1, 3, 1],
    'B': [1, 2, 3, 1],
    'C': [1, 2, 3, 4]
})

encoded_data, huffman_codes = huffman_encoding(data)

print("Encoded Data:")
print(encoded_data)
print("\nHuffman Codes:")
print(huffman_codes)

Encoded Data:
    A    B    C
0   0    0    0
1   0  111  111
2  10   10   10
3   0    0  110

Huffman Codes:
{1: '0', 3: '10', 4: '110', 2: '111'}


In [38]:
# iterate all elements of encoded_data and get the total length
total_length = 0
for col in encoded_data.columns:
  for val in encoded_data[col]:
    total_length += len(val)


In [41]:
encoded_data.astype(str).applymap(len).values.sum()

21

In [12]:
# Test DPCM
test_df = pd.DataFrame({
    '1':[1, 7, 3, 4],
    '2':[5, 9, 7, 8],
    '3':[9, 20, 11, 12],
    '4':[10, 10, 19, 16]
})

dpcm_df = DPCM(test_df)
print(dpcm_df)

     1    2     3     4
0  1.0  5.0   9.0  10.0
1  6.0  4.0  11.0   0.0
2 -4.0 -2.0  -9.0   9.0
3  1.0  1.0   1.0  -3.0


# Compress Bonn data

In [16]:
f_dpcm = DPCM(df_F)
o_dpcm = DPCM(df_O)
s_dpcm = DPCM(df_S)
f_encoded, f_codes = huffman_encoding(f_dpcm)
o_encoded, o_codes = huffman_encoding(o_dpcm)
s_encoded, s_codes = huffman_encoding(s_dpcm)

In [42]:
f_length = f_encoded.astype(str).applymap(len).values.sum()
o_length = o_encoded.astype(str).applymap(len).values.sum()
s_length = s_encoded.astype(str).applymap(len).values.sum()

cr_f = 4097*100*12 / f_length
cr_o = 4097*100*12 / o_length
cr_s = 4097*100*12 / s_length
print(f'Compression ratio for F: {cr_f}')
print(f'Compression ratio for O: {cr_o}')
print(f'Compression ratio for S: {cr_s}')

Compression ratio for F: 2.129795169015994
Compression ratio for O: 1.7762407744878008
Compression ratio for S: 1.397969867044433


dataset |  my result
----|---
F |  2.13
O |  1.78
S |  1.4



# Import and compress MIT data

In [43]:
os.chdir('/content/drive/My Drive/0-Project/MIT')

In [44]:
# 48 csv files
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

df_mix = pd.DataFrame()

for file in csv_files:
  file_path = os.path.join(os.getcwd(), file)

  df = pd.read_csv(file_path, skiprows=1)

  df_last_two_cols = df.iloc[:, -2:]

  file_prefix = file[:3]
  df_last_two_cols.columns = [f"{file_prefix}-1", f"{file_prefix}-2"]

  df_mix = pd.concat([df_mix, df_last_two_cols], axis=1)


In [54]:
def transfer(data):
  '''
  Params:
    data : input data frame
  First we add offset to input data frame so we have a dataset of non-negative numbers.
  Transfer the data using following equation:
  new_data = 4095 * data[i] / (max - min)
  '''
  new_data = data.copy()
  min = data.min().min()
  if min < 0:
    new_data -= min
  min = new_data.min().min()
  max = new_data.max().max()
  new_data = 4095 * new_data / (max - min)
  new_data = new_data.round().astype(int)
  return new_data

In [56]:
# first we scale all data
# df_mix_scaled = df_mix.mul(600)
# round the scaled result into integers
# df_mix_scaled = df_mix_scaled.round().astype(int)
# scaled_min = df_mix_scaled.min().min()
# # add offset to all data, make them non-negative
# df_mix_scaled = df_mix_scaled.add(abs(scaled_min))

df_mix_scaled = transfer(df_mix)



In [57]:
# choose 1,3,5.. columns in df_mixed_scaled as mixed_signal_1
mixed_signal_1 = df_mix_scaled.iloc[:,1::2]
# choose 0,2,4.. columns in df_mixed_scaled as mixed_signal_2
mixed_signal_2 = df_mix_scaled.iloc[:,::2]

In [59]:
mixed_1_dpcm = DPCM(mixed_signal_1)
mixed_2_dpcm = DPCM(mixed_signal_2)

In [60]:
mixed_1_encoded, mixed_1_codes = huffman_encoding(mixed_1_dpcm)
mixed_2_encoded, mixed_2_codes = huffman_encoding(mixed_2_dpcm)


In [61]:
mixed_1_length = mixed_1_encoded.astype(str).applymap(len).values.sum()
mixed_2_length = mixed_2_encoded.astype(str).applymap(len).values.sum()
cr_mixed_1 = 3600*48*12 / mixed_1_length
cr_mixed_2 = 3600*48*12 / mixed_2_length
print(f'Compression ratio for mixed_signal_1: {cr_mixed_1}')
print(f'Compression ratio for mixed_signal_2: {cr_mixed_2}')

Compression ratio for mixed_signal_1: 2.4353463462758085
Compression ratio for mixed_signal_2: 2.347834061936352


dataset | my result using scaling | my result using transfer
--|--|--
mixed signal 1 | 2.819 | 2.435
mixed signal 2 | 2.697 | 2.348

